In [16]:
from sqlalchemy import create_engine
import pandas as pd

from sqlalchemy import create_engine

def get_db_connection():
    username = input("Introduce tu usuario de MySQL: ")
    password = input("Introduce tu contraseña de MySQL: ")
    
    # Configuración de la base de datos
    host = 'localhost'  # Cambia a la dirección de tu servidor si no es local
    port = 3306         # Cambia el puerto si es diferente al predeterminado
    database = 'sakila'

    # Crear la conexión con SQLAlchemy
    try:
        engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')
        print("Conexión exitosa a la base de datos.")
        return engine
    except Exception as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None


In [8]:
def rentals_month(engine, month, year):
    query = f"""
    SELECT 
        customer_id, 
        COUNT(*) AS rentals 
    FROM 
        rental 
    WHERE 
        MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    GROUP BY 
        customer_id
    """
    df = pd.read_sql(query, engine)
    return df


In [10]:
def rental_count_month(df, month, year):

    new_column_name = f"rentals_{str(month).zfill(2)}_{year}"
    df = df.rename(columns={"rentals": new_column_name})
    return df


In [9]:
def compare_rentals(df1, df2):

    merged_df = pd.merge(df1, df2, on="customer_id", how="outer").fillna(0)
    
    rental_columns = [col for col in merged_df.columns if col.startswith('rentals')]
    merged_df['difference'] = merged_df[rental_columns[0]] - merged_df[rental_columns[1]]
    
    return merged_df


In [18]:
if __name__ == "__main__":

    engine = get_db_connection()

    may_rentals = rentals_month(engine, 5, 2005)
    june_rentals = rentals_month(engine, 6, 2005)

    may_rentals = rental_count_month(may_rentals, 5, 2005)
    june_rentals = rental_count_month(june_rentals, 6, 2005)

    comparison = compare_rentals(may_rentals, june_rentals)

    print(comparison)


Conexión exitosa a la base de datos.
     customer_id  rentals_05_2005  rentals_06_2005  difference
0              1              2.0              7.0        -5.0
1              2              1.0              1.0         0.0
2              3              2.0              4.0        -2.0
3              5              3.0              5.0        -2.0
4              6              3.0              4.0        -1.0
..           ...              ...              ...         ...
593          583              0.0              6.0        -6.0
594          585              0.0              4.0        -4.0
595          591              0.0              3.0        -3.0
596          592              0.0              5.0        -5.0
597          598              0.0              1.0        -1.0

[598 rows x 4 columns]
